In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
spark

In [4]:
hr_employee = spark.read.csv("file:///home/hadoop/Downloads/HR_Employee.csv",inferSchema=True,header=True)

In [6]:
hr_employee.printSchema()

root
 |-- EmployeeID: integer (nullable = true)
 |-- Department: string (nullable = true)
 |-- JobRole: string (nullable = true)
 |-- Attrition: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- MaritalStatus: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- EducationField: string (nullable = true)
 |-- BusinessTravel: string (nullable = true)
 |-- JobInvolvement: string (nullable = true)
 |-- JobLevel: integer (nullable = true)
 |-- JobSatisfaction: string (nullable = true)
 |-- Hourlyrate: integer (nullable = true)
 |-- Income: integer (nullable = true)
 |-- Salaryhike: integer (nullable = true)
 |-- OverTime: string (nullable = true)
 |-- Workex: integer (nullable = true)
 |-- YearsSinceLastPromotion: integer (nullable = true)
 |-- EmpSatisfaction: string (nullable = true)
 |-- TrainingTimesLastYear: integer (nullable = true)
 |-- WorkLifeBalance: string (nullable = true)
 |-- Performance_Rating: string (nul

## Big data File types and Optimization
1. compression
      * Parquet FileFormat = column wise storage of records, this file format compresses dataset,.csv or structured format into Parquet format.Parquet format will compress file format from original to compress by reducing filesize.
      * similar to this there are other file formats - orc,avro (stored records as keys and values format, schema format is stored as json format.

### write csv file to local directory as parquet file format

In [9]:
hr_employee.write.parquet('file:///home/hadoop/Downloads/HR/')

In [16]:
!hdfs dfs -rm -r /HR_data

Deleted /HR_data


In [10]:
spark.read.parquet('file:///home/hadoop/Downloads/HR/').show()

+----------+--------------------+--------------------+---------+------+---+-------------+-------------+--------------+-----------------+--------------+--------+---------------+----------+------+----------+--------+------+-----------------------+---------------+---------------------+---------------+------------------+
|EmployeeID|          Department|             JobRole|Attrition|Gender|Age|MaritalStatus|    Education|EducationField|   BusinessTravel|JobInvolvement|JobLevel|JobSatisfaction|Hourlyrate|Income|Salaryhike|OverTime|Workex|YearsSinceLastPromotion|EmpSatisfaction|TrainingTimesLastYear|WorkLifeBalance|Performance_Rating|
+----------+--------------------+--------------------+---------+------+---+-------------+-------------+--------------+-----------------+--------------+--------+---------------+----------+------+----------+--------+------+-----------------------+---------------+---------------------+---------------+------------------+
|         1|               Sales|     Sales

### writing and reading orc 


In [17]:
hr_employee.write.orc('/HR_data')

In [19]:
spark.read.orc('/HR_data').show()

+----------+--------------------+--------------------+---------+------+---+-------------+-------------+--------------+-----------------+--------------+--------+---------------+----------+------+----------+--------+------+-----------------------+---------------+---------------------+---------------+------------------+
|EmployeeID|          Department|             JobRole|Attrition|Gender|Age|MaritalStatus|    Education|EducationField|   BusinessTravel|JobInvolvement|JobLevel|JobSatisfaction|Hourlyrate|Income|Salaryhike|OverTime|Workex|YearsSinceLastPromotion|EmpSatisfaction|TrainingTimesLastYear|WorkLifeBalance|Performance_Rating|
+----------+--------------------+--------------------+---------+------+---+-------------+-------------+--------------+-----------------+--------------+--------+---------------+----------+------+----------+--------+------+-----------------------+---------------+---------------------+---------------+------------------+
|         1|               Sales|     Sales

#### Optimization Techniques
    * Optimization technique for optimizing spark jobs and significantly improve performance of spark running queries, spark Jobs.
    1. compression
    

### 2. Partitioning
    * divides data into smaller chunks of which can be processed in parallel.

In [20]:
partitioned_df = hr_employee.repartition(3)

In [21]:
partitioned_df.write.parquet("/HRPartition/")

### 3. cache or persist
    * memory location not accesed one cached it is faster and optmized

In [22]:
hr_employee.cache()

DataFrame[EmployeeID: int, Department: string, JobRole: string, Attrition: string, Gender: string, Age: int, MaritalStatus: string, Education: string, EducationField: string, BusinessTravel: string, JobInvolvement: string, JobLevel: int, JobSatisfaction: string, Hourlyrate: int, Income: int, Salaryhike: int, OverTime: string, Workex: int, YearsSinceLastPromotion: int, EmpSatisfaction: string, TrainingTimesLastYear: int, WorkLifeBalance: string, Performance_Rating: string]

In [23]:
from pyspark import StorageLevel
hr_employee.persist(StorageLevel.MEMORY_AND_DISK)

DataFrame[EmployeeID: int, Department: string, JobRole: string, Attrition: string, Gender: string, Age: int, MaritalStatus: string, Education: string, EducationField: string, BusinessTravel: string, JobInvolvement: string, JobLevel: int, JobSatisfaction: string, Hourlyrate: int, Income: int, Salaryhike: int, OverTime: string, Workex: int, YearsSinceLastPromotion: int, EmpSatisfaction: string, TrainingTimesLastYear: int, WorkLifeBalance: string, Performance_Rating: string]

### 4. Serialization

   #### *Efficient Serialization reduces time to read write data and transfer it over the network. Kyro Seriaalization is a popular serialization method over default serialization which is default java serialization.

#### a) Java Serialization: It is default serialization method,easy to use but drawback is it will slow down read and write process. It can produce large serialized sizes.

In [24]:
from pyspark import SparkConf,SparkContext
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Java Serialization Example").getOrCreate()

#### b) Kyro Serialization: This is faster more compact than Java Serialization

In [31]:
spark.stop()

In [32]:
"""
spark = SparkSession.builder\
.config("spark.serializer","org.apache.spark.serializer.KyroSerializer")\
.config("spark.kyro.registrationRequired","true")\
.config("sprk.kyro.classesToRegister","org.apache.spark.example.Person")\
.appName("Kyro Serialization Example").getOrCreate()
"""

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.ClassNotFoundException: org.apache.spark.serializer.KyroSerializer
	at java.net.URLClassLoader.findClass(URLClassLoader.java:381)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:424)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:357)
	at java.lang.Class.forName0(Native Method)
	at java.lang.Class.forName(Class.java:348)
	at org.apache.spark.util.Utils$.classForName(Utils.scala:238)
	at org.apache.spark.SparkEnv$.instantiateClass$1(SparkEnv.scala:259)
	at org.apache.spark.SparkEnv$.instantiateClassFromConf$1(SparkEnv.scala:280)
	at org.apache.spark.SparkEnv$.create(SparkEnv.scala:283)
	at org.apache.spark.SparkEnv$.createDriverEnv(SparkEnv.scala:175)
	at org.apache.spark.SparkContext.createSparkEnv(SparkContext.scala:257)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:424)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at

In [33]:
spark = SparkSession.builder.appName('PySparkSession')\
.config("spark.serializer","org.apache.spark.serializer.JavaSerializer").getOrCreate()

### 5. Broadcasting Variables
    * Broadcasting small datasets improvee join performance

In [40]:
small_df = spark.read.csv("file:///home/hadoop/Downloads/airports.csv", header=True, inferSchema=True)
df = spark.read.csv("file:///home/hadoop/Downloads/raw_flight_data.csv", header=True, inferSchema=True)


In [41]:
from pyspark.sql.functions import broadcast
broadcast_df = broadcast(small_df)

In [42]:
broadcast_df.cache()
df.cache()

DataFrame[DayofMonth: int, DayOfWeek: int, Carrier: string, OriginAirportID: int, DestAirportID: int, DepDelay: int, ArrDelay: int]

In [44]:
df.join(broadcast_df,df.OriginAirportID == broadcast_df.airport_id)

DataFrame[DayofMonth: int, DayOfWeek: int, Carrier: string, OriginAirportID: int, DestAirportID: int, DepDelay: int, ArrDelay: int, airport_id: int, city: string, state: string, name: string]

### 6. Level of Parallelism

In [45]:
# Adjust level of parallelism based on your cluster size
spark.conf.set("spark.default.parallelism",100)

### 7. avoid groupByKey: performs a lot of inbetween shuffling
    * instead reduceByKey or aggregateByKey() instead of it

In [46]:
rdd = spark.sparkContext.parallelize([('Dosa',2),('Salad',3),('Idli',1),('Dosa',3),('Chocolates',4),('Idli',2)])
rdd.reduceByKey(lambda x,y:x+y).collect()

[('Dosa', 5), ('Salad', 3), ('Idli', 3), ('Chocolates', 4)]

### 8. Reduce Shuffling
    *ensures that key will be passed to same worker node
    *reduces the number of shuffles by optimizing transformation
    *use reduceByKey() over groupByKey()
    *use map and reduce function over groupBy

### 9.Repartition and Coalesce

### 10.Accumulators
    * Use accumulators for aggregate information like count(),sum(),max() etc. accross executors

In [48]:
acc = spark.sparkContext.accumulator(0)

In [49]:
type(acc)

pyspark.accumulators.Accumulator

In [50]:
rdd = spark.sparkContext.parallelize([1,2,3,4,5,6,7,8])

In [51]:
# python user defined function
def add(x):
    acc.add(x)

In [52]:
rdd.foreach(add)

In [55]:
print(acc.value)

36


In [77]:
unique_days_df = df.select('DayOfWeek').distinct()


In [79]:
unique_days_df.show()

+---------+
|DayOfWeek|
+---------+
|        1|
|        6|
|        3|
|        5|
|        4|
|        7|
|        2|
+---------+



### 11.Bucketing
    * used to create bucket of large datasets for efficient query and joins

In [72]:
df.write.bucketBy(7,"DayOfWeek").saveAsTable("bucket_table")

In [ ]:
# !hdfs dfs -cat //locaction
# this runs the part0000 file in browser